# **NAML MODEL**

### **Initial Setup**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
route = '/content/drive/MyDrive/IMPACT PROJECT'

%cd {route}/recommenders

/content/drive/MyDrive/IMPACT PROJECT/recommenders


In [3]:
!pip install scrapbook retrying

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.2 MB/s eta 0:00:00


###  **Importing the needed libraries**

In [4]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.naml import NAMLModel
from recommenders.models.newsrec.io.mind_all_iterator import MINDAllIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]
Tensorflow version: 2.12.0


### **Loading the behaviours and news dataframes**

In [5]:
# Options: demo, small, large
MIND_type = 'demo'

In [6]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding_all.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict_all.pkl")
vertDict_file = os.path.join(data_path, "utils", "vert_dict.pkl")
subvertDict_file = os.path.join(data_path, "utils", "subvert_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'naml.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)

if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

100%|██████████| 17.0k/17.0k [00:01<00:00, 10.6kKB/s]
100%|██████████| 9.84k/9.84k [00:01<00:00, 8.88kKB/s]
100%|██████████| 95.0k/95.0k [00:03<00:00, 25.4kKB/s]


### **NAML: Neural News Recommendation with Attentive Multi-View Learning**

Setting up the parameters

In [7]:
epochs = 5
seed = 42
batch_size = 32

In [8]:
hparams = prepare_hparams(yaml_file,
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file,
                          userDict_file=userDict_file,
                          vertDict_file=vertDict_file,
                          subvertDict_file=subvertDict_file,
                          batch_size=batch_size,
                          epochs=epochs)
print(hparams)

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 4, 'head_dim': 100, 'filter_num': 400, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 5, 'batch_size': 32, 'show_step': 100000, 'title_size': 30, 'body_size': 50, 'his_size': 50, 'vert_num': 17, 'subvert_num': 249, 'data_format': 'naml', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'cnn_activation': 'relu', 'model_type': 'naml', 'dense_activation': 'relu', 'loss': 'cross_entropy_loss', 'wordEmb_file': '/tmp/tmpcbs3c8wh/utils/embedding_all.npy', 'wordDict_file': '/tmp/tmpcbs3c8wh/utils/word_dict_all.pkl', 'userDict_file': '/tmp/tmpcbs3c8wh/utils/uid2index.pkl', 'vertDict_file': '/tmp/tmpcbs3c8wh/utils/vert_dict.pkl', 'subvertDict_file': '/tmp/tmpcbs3c8wh/utils/subvert_dict.pkl'}


Instantiating the model

In [9]:
iterator = MINDAllIterator

In [12]:
model = NAMLModel(hparams, iterator, seed=seed)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Training the NAML model

In [13]:
%%time
model.fit(train_news_file, train_behaviors_file,valid_news_file, valid_behaviors_file)

1085it [03:27,  5.23it/s]
0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
18693it [00:34, 539.80it/s]
7507it [00:15, 474.44it/s]
7538it [00:01, 6841.06it/s]


at epoch 1
train info: logloss loss:1.488674212161297
eval info: group_auc:0.5757, mean_mrr:0.2491, ndcg@10:0.337, ndcg@5:0.2692
at epoch 1 , train time: 207.6 eval time: 60.1


1085it [03:12,  5.62it/s]
18693it [00:33, 565.50it/s]
7507it [00:14, 508.15it/s]
7538it [00:01, 6443.00it/s]


at epoch 2
train info: logloss loss:1.411944036549687
eval info: group_auc:0.6193, mean_mrr:0.2842, ndcg@10:0.3759, ndcg@5:0.3147
at epoch 2 , train time: 192.9 eval time: 57.5


1085it [03:13,  5.62it/s]
18693it [00:33, 565.50it/s]
7507it [00:14, 506.27it/s]
7538it [00:01, 7226.53it/s]


at epoch 3
train info: logloss loss:1.3707006801657962
eval info: group_auc:0.6249, mean_mrr:0.2808, ndcg@10:0.3751, ndcg@5:0.309
at epoch 3 , train time: 193.1 eval time: 57.5


1085it [03:12,  5.62it/s]
18693it [00:33, 563.81it/s]
7507it [00:14, 507.20it/s]
7538it [00:00, 7603.76it/s]


at epoch 4
train info: logloss loss:1.3418200905971263
eval info: group_auc:0.6299, mean_mrr:0.2862, ndcg@10:0.3792, ndcg@5:0.3147
at epoch 4 , train time: 193.0 eval time: 57.5


1085it [03:13,  5.62it/s]
18693it [00:33, 561.47it/s]
7507it [00:15, 498.62it/s]
7538it [00:00, 7638.45it/s]


at epoch 5
train info: logloss loss:1.3199062233696335
eval info: group_auc:0.6187, mean_mrr:0.2848, ndcg@10:0.3736, ndcg@5:0.3098
at epoch 5 , train time: 193.0 eval time: 57.9
CPU times: user 13min 53s, sys: 2min 47s, total: 16min 40s
Wall time: 21min 10s


Evaluating the NAML Model

In [14]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

18693it [00:33, 562.57it/s]
7507it [00:14, 506.76it/s]
7538it [00:00, 8096.47it/s]


{'group_auc': 0.6187, 'mean_mrr': 0.2848, 'ndcg@5': 0.3098, 'ndcg@10': 0.3736}
CPU times: user 1min 47s, sys: 24 s, total: 2min 11s
Wall time: 57.5 s


| Model   | group_auc | mean_mrr | ndcg@5 | ndcg@10 |
|----------|-----------|----------|--------|---------|
| NAML    |   0.6187  |   0.2848  | 0.3098 |  0.3736 |

In [15]:
sb.glue("res_syn", res_syn)

Saving the NMRS Model

In [16]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "naml_ckpt"))

--------------